In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

C:\Users\Hano\AppData\Local\Temp\ipykernel_37692\3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython.display
  from IPython.core.display import display, HTML


# Lab | Natural Language Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [2]:
! pip install nltk


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\Hano\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
import nltk
nltk.download('all')


In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import nltk
import re
from nltk.corpus import stopwords 
from nltk.stem import WordNetLemmatizer
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [6]:
## Read Data for the Fraudulent Email Kaggle Challenge
## ../ go up of the file 
data = pd.read_csv("../data/kg_train.csv",encoding='latin-1')

# Reduce the training set to speed up development. 
# Modify for final system
data = data.head(1000)
print(data.shape)
data.fillna("",inplace=True)

(1000, 2)


### Let's divide the training and test set into two partitions

In [7]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

## Data Preprocessing

In [ ]:
import string
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
# punctuation   
print(string.punctuation)
#stopwords  
print(stopwords.words("english")[100:110])
snowball = SnowballStemmer('english')

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on']


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [10]:
# Your code
import re


In [11]:

def clean_html(html_text):
     
    soup = BeautifulSoup(html_text, "html.parser")
    
    
    for script in soup(["script", "style"]):
        script.decompose()  
        
    
    comments = soup.find_all(string=lambda text: isinstance(text, Comment))
    for comment in comments:
        comment.extract()  
        
    cleaned_text = soup.get_text()  
    
    
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)  
    cleaned_text = cleaned_text.strip()  
    
    return cleaned_text

- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [12]:
# Your code
def clean_text(text):
    
    text = text.lower()
    text = re.sub(r"\bb\b", "", text)
    text = re.sub(r"[^a-zA-Z\s]", "", text)
    text = re.sub(r"\d+", "", text)
    text = re.sub(r"\b\w\b", "", text)
    text = re.sub(r"^\w\s", "", text)
    text = re.sub(r"\s+", " ", text)
    text = text.strip()
    
    return text

## Now let's work on removing stopwords
Remove the stopwords.

In [13]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [14]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Hano\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Hano\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [15]:
# Your code
def remove_stopwords(text):
    
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(text)
    filtered_words = [word for word in words if word.lower() not in stop_words]
    cleaned_text = ' '.join(filtered_words)
    
    return cleaned_text

## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [16]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer


In [17]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')  # Open Multilingual Wordnet (يدعم اللغات المتعددة)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Hano\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Hano\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Hano\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [18]:
lemmatizer = WordNetLemmatizer()

In [19]:
# Your code
def lemmatize_text(text):
    
    words = word_tokenize(text)
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    lemmatized_text = ' '.join(lemmatized_words)
    
    return lemmatized_text

## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [20]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

In [21]:


# Sample data
data = pd.DataFrame({
    'text': ["This is a sample sentence.", "Another example sentence for testing."],
    'label': [1, 0]
})

# Splitting the data into training and test sets
X = data['text']
y = data['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the CountVectorizer
vectorizer = CountVectorizer()

# Fit the vectorizer on the training data and transform both training and test data
X_train_bow = vectorizer.fit_transform(X_train)
X_test_bow = vectorizer.transform(X_test)

# Convert the result to a DataFrame for easier visualization
X_train_bow_df = pd.DataFrame(X_train_bow.toarray(), columns=vectorizer.get_feature_names_out())
X_test_bow_df = pd.DataFrame(X_test_bow.toarray(), columns=vectorizer.get_feature_names_out())

# Display the Bag of Words representation
print("Training Data BoW:")
print(X_train_bow_df.head())

print("\nTest Data BoW:")
print(X_test_bow_df.head())


Training Data BoW:
   is  sample  sentence  this
0   1       1         1     1

Test Data BoW:
   is  sample  sentence  this
0   0       0         1     0


In [22]:
print(data.columns)

Index(['text', 'label'], dtype='object')


## Extra features

In [23]:

data_train = pd.DataFrame({
    'preprocessed_text': ["This is a test with euro and dollar.", "Another example with free money and transfer."]
})

data_val = pd.DataFrame({
    'preprocessed_text': ["Bank account and transaction.", "Cheap deals and win big!"]
})

In [24]:
# We add to the original dataframe two additional indicators (money symbols and suspicious words).
money_simbol_list = "|".join(["euro","dollar","pound","€",r"\$"])
suspicious_words = "|".join(["free","cheap","sex","money","account","bank","fund","transfer","transaction","win","deposit","password"])

data_train['money_mark'] = data_train['preprocessed_text'].str.contains(money_simbol_list)*1
data_train['suspicious_words'] = data_train['preprocessed_text'].str.contains(suspicious_words)*1
data_train['text_len'] = data_train['preprocessed_text'].apply(lambda x: len(x)) 

data_val['money_mark'] = data_val['preprocessed_text'].str.contains(money_simbol_list)*1
data_val['suspicious_words'] = data_val['preprocessed_text'].str.contains(suspicious_words)*1
data_val['text_len'] = data_val['preprocessed_text'].apply(lambda x: len(x)) 

data_train.head()

,preprocessed_text,money_mark,suspicious_words,text_len
0,This is a test with euro and dollar.,1,0,36
1,Another example with free money and transfer.,1,1,45


## How would work the Bag of Words with Count Vectorizer concept?

In [25]:
# Your code

data = pd.DataFrame({
    'text': ["This is a sample sentence.", "Another example sentence for testing.", "Free money in your bank account.", "Transfer funds to win big prizes!"]
})

# Splitting the data into training and test sets
X_train, X_test = train_test_split(data['text'], test_size=0.2, random_state=42)

# Initialize the CountVectorizer
vectorizer = CountVectorizer()

# Fit the vectorizer on the training data and transform both training and test data
X_train_bow = vectorizer.fit_transform(X_train)
X_test_bow = vectorizer.transform(X_test)

# Convert the result to a DataFrame for easier visualization
X_train_bow_df = pd.DataFrame(X_train_bow.toarray(), columns=vectorizer.get_feature_names_out())
X_test_bow_df = pd.DataFrame(X_test_bow.toarray(), columns=vectorizer.get_feature_names_out())

# Display the Bag of Words representation
print("Training Data BoW:")
print(X_train_bow_df.head())

print("\nTest Data BoW:")
print(X_test_bow_df.head())

Training Data BoW:
   account  bank  big  free  funds  in  is  money  prizes  sample  sentence  \
0        0     0    1     0      1   0   0      0       1       0         0   
1        0     0    0     0      0   0   1      0       0       1         1   
2        1     1    0     1      0   1   0      1       0       0         0   

   this  to  transfer  win  your  
0     0   1         1    1     0  
1     1   0         0    0     0  
2     0   0         0    0     1  

Test Data BoW:
   account  bank  big  free  funds  in  is  money  prizes  sample  sentence  \
0        0     0    0     0      0   0   0      0       0       0         1   

   this  to  transfer  win  your  
0     0   0         0    0     0  


## TF-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [26]:
# Your code


data = pd.DataFrame({
    'text': [
        "This is a sample sentence.",
        "Another example sentence for testing.",
        "Free money in your bank account.",
        "Transfer funds to win big prizes!"
    ]
})

# Initialize the TfidfVectorizer
vectorizer = TfidfVectorizer()

# Fit the vectorizer on the entire dataset and transform it into TF-IDF features
tfidf_matrix = vectorizer.fit_transform(data['text'])

# Print the shape of the vectorized dataset
print("Shape of the vectorized dataset:", tfidf_matrix.shape)


Shape of the vectorized dataset: (4, 20)


## And the Train a Classifier?

In [ ]:
# Your code

X_train, X_test, y_train, y_test = train_test_split(tfidf_matrix, data['label'], test_size=0.2, random_state=42)

classifier = MultinomialNB()
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Accuracy:", accuracy)
print("\nClassification Report:")
print(report)


### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to **find the most relevant features**.

For example, you can test the following options and check which of them performs better:
- Using "Bag of Words" only
- Using "TF-IDF" only
- Bag of Words + extra flags (money_mark, suspicious_words, text_len)
- TF-IDF + extra flags


You can work with teams of two persons (recommended).

In [ ]:
# Your code